In [25]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy import stats

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


# Getting Dataframes

In [26]:
master_data = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
data = master_data
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [27]:
master_submit = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
submit = master_submit
submit.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


### Data Wrangling

In [28]:
data.shape

(1460, 81)

In [29]:
submit.shape

(1459, 80)

In [30]:
#data.info(verbose=True)

In [31]:
#pd.set_option('display.max_rows',None)
#data.isna().sum()

In [32]:
#data['MiscVal'].unique()

In [33]:
#data[['MiscFeature','MiscVal']]

In [34]:
data = data.drop(['MiscFeature'], axis=1)
submit = submit.drop(['MiscFeature'], axis=1)

In [35]:
print("shape of given data", data.shape)
print("shape of submit data", submit.shape)

shape of given data (1460, 80)
shape of submit data (1459, 79)


# Feature Engineering

## Separating independent and dependent variables

In [36]:
X_data=data
X_data=X_data.drop(['SalePrice'], axis=1)
X_submit=submit
#X_data.head()

In [37]:
Y_data=data['SalePrice']
Y_data.head()

0    208500
1    181500
2    223500
3    140000
4    250000
Name: SalePrice, dtype: int64

## One hot encoding

In [38]:
print("shape before one hot encoding", X_data.shape)
X_data=pd.get_dummies(X_data)

dummies_frame = pd.get_dummies(X_data)
X_submit=X_submit.reindex(columns = dummies_frame.columns, fill_value=0)
#X_test=pd.get_dummies(X_test)

print("shape after one hot encoding", X_data.shape)
X_data.head()

shape before one hot encoding (1460, 79)
shape after one hot encoding (1460, 285)


,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,0,0,0,1,0,0,0,0,1,0
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,0,0,0,1,0,0,0,0,1,0
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,0,0,0,1,0,0,0,0,1,0
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,0,0,0,1,1,0,0,0,0,0
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,0,0,0,1,0,0,0,0,1,0


In [39]:
X_data=X_data.fillna(0)
X_submit=X_submit.fillna(0)
#X.isna().sum()

In [40]:
print("shape of X_data", X_data.shape)
print("shape of X_submit", X_submit.shape)

shape of X_data (1460, 285)
shape of X_submit (1459, 285)


## Train Test Split

## Data Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled_data = scaler.fit_transform(X_data)
X_scaled_test = scaler.transform(X_test)
X_scaled_train

In [ ]:
np.any(np.isnan(X_scaled_train))

## Dimensionality Reduction using PCA

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(0.95)
X_pca = pca.fit_transform(X_scaled_train)
X_pca.shape

# Ridge Regression

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV

errors=[]

sweep = np.arange(1, 5, 1, dtype=int)
for i in sweep:
    poly = PolynomialFeatures(degree = 1)
    X_poly=poly.fit_transform(X_scaled_train)
    ridge = RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1], cv=4)
    ridge.fit(X_poly, Y_train)
    errors.append(ridge.score(X_poly, Y_train))
    
print(errors)

## Submission

In [ ]:
predictions = ridge.predict(X_scaled_test)
print(predictions)

In [ ]:
X_test['SalePrice']=predictions
#X_test.head()

In [ ]:
submit = X_test[['Id','SalePrice']].set_index('Id')
Id = X_test[["Id"]].to_numpy()
SalePrice = X_test[["SalePrice"]].to_numpy()
submit.head()

In [ ]:
!cd /kaggle/working
submit.to_csv("submission.csv")
!head submission.csv

In [ ]:
SalePrice.shape